BASES DE DADOS UTILIZADAS:
- Web of Science
- Scopus

TERMO DE BUSCA UTILIZADOS:
- (ceramic OR calcium phosphate) AND (piezoelectric coefficient OR mechanic OR elastic coefficient OR polarization)


In [ ]:
#!pip install xlrd
#!pip install regex
#!pip install google-generativeai
import pandas as pd
import regex as re
import os

diretorio = "/content/drive/MyDrive/DOUTORADO/DISCIPLINAS/CIENCIA DE DADOS PARA MATERIAIS/NOTEBOOKS/Relatorio 02 - NLP/CaP-properties"

dados_total = pd.DataFrame()

for file in os.listdir(diretorio):
    if file.endswith('.xls'):
      dados = pd.read_excel(f"{diretorio}/{file}")
      dados_total = pd.concat([dados_total, dados], ignore_index=True)

dados_total.dropna(subset=['Abstract'])

raw_abstracts = list()
for abst in dados_total['Abstract']:
    raw_abstracts.append(str(abst))

len(raw_abstracts)


2717

FILTRA APENAS AQUELES ABSTRACTS QUE TENHAM ALGUMA DOS VALORES DE INTERESSE

In [ ]:
i = 0
abstracts = list()
no_data_abstracts = list()
for abstract in raw_abstracts:
    i += 1
    print('Processando Abstract', i)
    #print(abstract)
    if re.search(r'[TGM]?Pa', abstract) or re.search(r'C/[cm]?m', abstract) or re.search(r'C/N', abstract):
      abstracts.append(abstract)
    else:
      no_data_abstracts.append(abstract)
      print('====>>> The abstract does not have the target properties. <<<=====')

print("Quantidade de artigos que serão analisados: ", len(abstracts))
print("Quantidade de artigos que não contem informaçõe mínimas: ", len(no_data_abstracts))

Processando Abstract 1
====>>> The abstract does not have the target properties. <<<=====
Processando Abstract 2
====>>> The abstract does not have the target properties. <<<=====
Processando Abstract 3
====>>> The abstract does not have the target properties. <<<=====
Processando Abstract 4
====>>> The abstract does not have the target properties. <<<=====
Processando Abstract 5
====>>> The abstract does not have the target properties. <<<=====
Processando Abstract 6
====>>> The abstract does not have the target properties. <<<=====
Processando Abstract 7
====>>> The abstract does not have the target properties. <<<=====
Processando Abstract 8
====>>> The abstract does not have the target properties. <<<=====
Processando Abstract 9
====>>> The abstract does not have the target properties. <<<=====
Processando Abstract 10
Processando Abstract 11
====>>> The abstract does not have the target properties. <<<=====
Processando Abstract 12
====>>> The abstract does not have the target prope

ANALISE DE ABSTRACTS QUE CONTENHAM DADOS DE ALGUMA MEDIDA DE INTERESSE

In [ ]:
# To run this code you need to install the following dependencies:
#!pip install google-genai

import base64
import os
from google import genai
from google.genai import types
from google.colab import userdata
googleai_key = userdata.get('AIGOOGLE_KEY')


def generate(abstract):
    client = genai.Client(
        # NOTE: It's highly recommended to use os.environ.get("GEMINI_API_KEY")
        # instead of hardcoding the key for security.
        api_key=googleai_key, # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< INSERT THE API KEY HERE!!!!!!!!!!!!!!!!!!!!!!!!!!!
        #api_key="teste_googleai",
    )

    model = "gemini-flash-latest"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text="""You want to find the mechanical, piezoelectric and polarization properties of ceramic materials, focusing on calcium phosphates.
                Extract mechanical, piezoelectric and polarization properties from the abstract with their corresponding ceramic composition.
                Output the results should be in a json format, with the key: value structure.
                Each material should be in a separate json.
                For each mechanical, piezoelectric and polarization property you should have a key, with the extracted quantity as a value.
                Check this example for an extraction with two compositions of materials (AlCoCuNiBa and AlNbCuNiPt):
                {
                    "AlCoCuNiBa" : {
                        "Yield Strenght": "100 MPa",
                        "Bulk Modulus": "10 GPa",
                        "Compressive strength": "1 GPa",
                        "Elastic coefficients": {
                          "C11": "400",
                          "C22": "200",
                          "C33": "300"
                        },
                        "Piezoelectric coefficients": {
                          "d11": "1 C/m²",
                          "d22": "0.5 C/m²",
                          "d33": "2 C/m²"
                        },
                        "Polarization": "0.5 mC/m²"
                    },
                    "AlNbCuNiPt" : {
                        "Yield Strenght": "10 MPa",
                        "Bulk Modulus": "1 GPa",
                        "Compressive strength": "10 GPa",
                        "Elastic coefficients": {
                          "C11": "150",
                          "C22": "210",
                          "C33": "410"
                        },
                        "Piezoelectric coefficients": {
                          "d11": "-1.3 C/m²",
                          "d22": "1.4 C/m²",
                          "d33": "3 C/m²"
                        },
                        "Polarization": "0.7 C/m²"
                    }
                }
                Now, if you don't find material composition, the output should be:
                {
                    "None": "None"
                }
                If you find the material, but no mechanical property nor piezoelectric property nor polarization data, it should be:
                {
                    "AlNbCuNiPt": "None"
                }
                If you find the material, with piezoelectric property and polarization data but no mechanical property, it should be:
                {
                    "AlNbCuNiPt": {
                        "Mechanical properties": "None",
                        "Piezoelectric coefficients": {
                          "d11": "-1.3 C/m²",
                          "d22": "1.4 C/m²",
                          "d33": "3 C/m²"
                        },
                        "Polarization": "0.7 C/m²"
                    }
                }
                If you find the material, with some of the target properties, for example, you find only polarization property, it should be:
                "AlNbCuNiPt" : {
                        "Mechanical properties": "None",
                        "Piezoelectric coefficients": "None",
                        "Polarization": "0.7 C/m²"
                    }

                Rules:
                - Ignore information unrelated to mechanical properties, piezoelectric properties or polarization data.
                - If information is unclear or there are more ceramics than properties, output None.
                - Prefer precise extractions over quantity.
                - Only extract values that have clear composition-property relationships.
                - Follow these instruction precisely!
                """),
            ],
        ),
        types.Content(
            role="user",
            # Use an f-string by starting the triple quotes with 'f'
            parts=[
                types.Part.from_text(text=f"""
                Analyze the following abstract text and extract the mechanical, piezoelectric and polarization properties according to the instructions above:

                **ABSTRACT TEXT START**
                {abstract}
                **ABSTRACT TEXT END**
                """),
            ],
        ),
    ]

    #print(contents)

    generate_content_config = types.GenerateContentConfig(
        max_output_tokens=8000,
        thinking_config = types.ThinkingConfig(
            thinking_budget=-1,
        ),
        # Removed image_config as it's not relevant for text-only tasks
        response_mime_type="application/json",
        system_instruction=[
            types.Part.from_text(text="""You are a bot mining for mechanical, piezoelectric and polarization properties of ceramics materials from literature's abstracts.
            You will receive detailed information on the format you will need to output the extracted information.
            Do not deviate from the instructions. Your output should only have the information on the requested format, nothing else."""),
        ],
    )

    # 1. Call the API to get the full response object.
    response = client.models.generate_content(
        model=model,
        contents=contents,
        config=generate_content_config,
    )

    # 2. Return ONLY the text content of the response, which will be your JSON string.
    return response.text

In [ ]:
"""Nota: If you find the material and there is some information that indicates the properties values were studied and can be find in the main paper,
                but no property value is clear on the abstract, it should be:
                "AlNbCuNiPt" : {
                        "Mechanical properties": "Possible",
                        "Piezoelectric coefficients": "Not present",  # if no indication of the piezoelectric properties was studied
                        "Polarization": "Possible"
                    }
"""

In [ ]:
range_analise = [0,100]
outputs = list()
i = 0

for abstract in abstracts[range_analise[0]:range_analise[1]]:
    print('Processing Abstract', i)
    print(abstract)
    response = None
    if re.search(r'[TGM]?Pa', abstract) or re.search(r'C/[cm]?m', abstract) or re.search(r'C/N', abstract):
        try:
            #print('========= POSSUI ALGUM DADO ======')
            response = generate(abstract)
            outputs.append(response)
            #print(response)
        except Exception as e:
            print(f'API Response Error: {str(e)}')
            outputs.append("Error")
    else:
        print('The abstract does not have the target properties.')
        outputs.append("None")
    i += 1

Processing Abstract 0
The appropriate immune microenvironment plays crucial roles in bone regeneration. Surface structure and chemistry are key factors affecting immune cells behavior and subsequently regulating the activity of bonerelated cells. To achieve rapid osteointegration, this study prepared a biomimetic calcium phosphate (CaP) coating on Ti-based substrates (THCaP) with the help of oleic acid under hydrothermal condition. The coating was composed of hydroxyapatite nanowires and further self-assembled into micropores. Also, the effect of the biomimetic CaP coating on the immune response of macrophages and the impact on angiogenesis and osteogenesis were investigated. In vitro cell culture results showed that compared with pristine Ti and similar titanate nanowire structure (TH), THCaP not only stimulated the polarization of macrophages towards a pro-healing M2 phenotype, but also enhanced the angiogenic and osteogenic differentiation of endothelial cells and preosteoblastic ce

Processing Abstract 1
Elastic moduli parameters and photon attenuation competence of phosphate (P2O5)/sodium (Na2O)/calcium (CaO) glasses with form 45P(2)O(5)-(55-x)Na2O-xCaO: x = 0-55 mol% (PNC-0:PNC-55) have been estimated and clarified. The development in Poisson's ratio and elastic moduli parameters (bulk, Young's, shear, and longitudinal) of the heavier CaO content of the PNC-glasses were proved through Makishima-Mackenzie (M-M) and bond compression (B-C) models. The B-C bulk elastic (KB-C) modulus was found to raise from 71.4 to 95.16 GPa, while the M-M bulk (KM-M) modulus was increased from 22.48 to 40.75 GPa for PNC-0 (glass sample free with CaO) and PNC-55 (glass sample rich with CaO), respectively. The highest mass attenuation coefficient (MAC) values were reported for PNC-55 sample, whereas the lowest were reported for PNC-0 sample. Half value layer (HVL) values were reported as 1.6526 cm, 1.6181 cm, 1.5734 cm, 1.5275 cm, 1.4847 cm, 1.441 cm, and 1.4172 cm for PNC-0, PNC-10,

Processing Abstract 3
Implant devices are prone to a substantial risk of infection, which can result in serious health issues with an increased number of fatalities sometimes. Numerous studies have demonstrated that calcium phosphate-based materials exhibit enhanced bone integration when subjected to electrical polarization through an external electric field prior to implantation The present work focuses on examining the impact of electrical polarization on a biphasic calcium phosphate consisting of 40% hydroxyapatite and 60% beta-tricalcium phosphate. To evaluate the osteogenicity and anti-bacterial efficacy, two types of specimens such as (i) that was not subjected to poling but were doped with silver nanoparticles, and (ii) that underwent poling without doping. Comprehensive examinations utilizing X-ray diffraction and scanning electron microscopy were used to assess the crystallinity and purity of the phase and to observe the cell morphology. The Thermally Stimulated Depolarization

Processing Abstract 4
Zinc (Zn) alloys provide a new generation for orthopedic applications due to their essential physiological effects and promising degradation properties. However, excessive release of Zn ions (Zn2+) during degradation and the severe inflammatory microenvironment are not conducive to osseointegration, which is determined by the characteristics of the implant surface. Therefore, it is essential to modulate the release rate of Zn alloys by surface modification technology and endow them with anti-inflammatory and osteogenic effects. In this study, two kinds of phosphate chemical conversion (PCC) coatings with different compositions and morphological structures are prepared, namely Zn-P (with disk-like crystals) and Ca-Zn-P (with lamellar crystals). Although all the PCC-coated Zn implants have low cytotoxicity, Ca-Zn-P show better osteoimmunomodulation effects in several aspects: the induction of the M2-phenotype macrophage polarization and thus promotion of osteogenesi

Processing Abstract 5
Hopeite (Zn3(PO4)24H2O) coatings, fabricated via zinc phosphate chemical conversion (ZPCC), have attracted considerable interest in biomedical applications owing to their excellent corrosion resistance and biocompatibility. However, the influence of calcium nitrate (CN) on coating properties remains poorly understood. This study systematically investigates the effect of CN concentration on the microstructure and corrosion behavior of ZPCC coatings deposited on stainless steel (SS). The phase composition, surface morphology, and elemental distribution were characterized using X-ray diffraction (XRD) and scanning electron microscopy with energy-dispersive spectroscopy (SEM-EDS). Electrochemical corrosion performance was assessed via potentiodynamic polarization in a 0.9 wt.% NaCl solution. The results indicate that CN concentration critically influences coating morphology, with higher concentrations leading to reduced crystal size and increased coating mass. Notably

...

Processing Abstract 62
In this study we compare three droplet microreactors for the synthesis of cerium oxide nanoparticles (CONs) and calcium phosphate crystals (CaPs) and discuss their suitability for long-term, continuous operation. The synthesis conditions were controlled by manipulating the flow rate ratio of the continuous phase to the dispersed (Qc/Qd), which is a crucial factor influencing droplet formation rate (DF) and droplet size. After identifying appropriate operating conditions (droplet size between 400 and 500 mu m), synthesis experiments were conducted to test the continuous operation of the microreactors. Among the three designs, one flow focusing device with a buffer stream, demonstrated superior resilience to particle fouling, and seemed to be appropriate for long-term, continuous particle synthesis. Particles synthesized using microfluidics were consistently smaller and exhibited less crystallinity than those produced during conventional batch synthesis due to the 

Processing Abstract 69
Purpose - Magnesium alloys, although valuable, are reactive and require protection before its application in many fields. The purpose of this study was to evaluate a novel anticorrosive chemical conversion film on AZ80 magnesium alloy by environmental-friendly calcium series surface pretreatment.Design/methodology/approach - The corrosion resistance of the film was evaluated by potentiodynamic polarization and electrochemical impedance spectroscopy in 3.5 Wt.% NaCl solution. The surface morphologies, microstructure and composition of the film were investigated by scanning electron microscopy and energy-dispersive spectroscopy.Findings - The corrosion current density of the calcium series film decreased by more than one order of magnitude as compared to that of the AZ80 magnesium alloy. The conversion film presented dry-mud morphology, and its thickness was estimated to be approximately 4 mu m. The conversion film was highly hydrophilic, and the organic coating ad

Processing Abstract 70
The biomaterial poly(l-lactic acid) (PLLA) is commonly used for bone fixation devices and dental surgeries; however its natural osseointegrating ability is generally poor, which can lead to dislocation and partial fractures. Recently, calcium phosphate bioceramics have been incorporated into PLLA in order to redress this issue and improve the osseointegrating ability of the material. This study incorporated calcium phosphate microbeads in PLLA at various concentrations and measured their effect on the critical stress intensity factor, K-IC and critical energy release rate at crack initiation, G(in), as well as the unusual fracture mechanic they induced. The inclusion of microbeads into the polymer matrix reduced the materials' fracture toughness, from a K-IC of 34 +/- 7 Pa m(-1/2) for blank PLLA, to 18 +/- 1 Pa m(-1/2) for the strongest bead containing group; and from a G(in) of 1030 +/- 150 J m(-2) for blank PLLA to 200 +/- 18 J m(-2) for the same microbead cont

Processing Abstract 71
Heterotopic ossification (HO) is a double-edged sword. Pathological HO presents as an undesired clinical complication, whereas controlled heterotopic bone formation by synthetic osteoinductive materials shows promising therapeutic potentials for bone regeneration. However, the mechanism of material-induced heterotopic bone formation remains largely unknown. Early acquired HO being usually accompanied by severe tissue hypoxia prompts the hypothesis that hypoxia caused by the implantation coordinates serial cellular events and ultimately induces heterotopic bone formation in osteoinductive materials. The data presented herein shows a link between hypoxia, macrophage polarization to M2, osteoclastogenesis, and material-induced bone formation. Hypoxia inducible factor-1 alpha (HIF-1 alpha), a crucial mediator of cellular responses to hypoxia, is highly expressed in an osteoinductive calcium phosphate ceramic (CaP) during the early phase of implantation, while pharmac

Processing Abstract 79
Calcium phosphate cement (CPC) is a widely used bone substitute in the clinic; however, the low strength of CPC limits its utilization. In this study, we investigated mechanical influences of chitosan fiber combined with gelatin on CPC, and examined the biocompatibility of the new composite with rat bone marrow stromal cells. Compared to the fiber impregnated in phosphate buffered saline (80.5 MPa), our study showed that tensile strength of chitosan fiber increased 106 and 114% with the impregnation of gelatin at the mass fraction 5 and 10%, although this increase was not statistically significant. It was demonstrated by Fourier transform infrared spectroscopy that the characteristic absorption bands of chitosan were changed with the addition of gelatin. The optimal flexural strength enhancement was obtained when CPC was reinforced with fiber at volume fraction of 30% and gelatin at mass fraction of 5% (maximum: 12.31 MPa). The fiber morphology was more compact w

In [ ]:
with open(f'{diretorio}/outputs.txt', 'w') as f:
    for output in outputs:
        f.write(f'{output}\n')
        print(f'{output}\n')